In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import re

%matplotlib inline

In [2]:
df = pd.read_csv("nlpdata.txt",sep=",,,",header=None ,names=['question','type'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
df.head()

,question,type
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [4]:
df.shape

(1483, 2)

In [5]:
df['type']=df['type'].str.strip()

In [6]:
df['type'].unique()

array(['unknown', 'what', 'when', 'who', 'affirmation'], dtype=object)

In [7]:
df['question'].values

array(['how did serfdom develop in and then leave russia ? ',
       'what films featured the character popeye doyle ? ',
       "how can i find a list of celebrities ' real names ? ", ...,
       'does this hose have one ? ', 'can i get it in india ? ',
       'would this work on a 2008 ford edge with a naked roof ? '],
      dtype=object)

In [8]:
df['question'] = df['question'].apply(lambda x: x.lower())
df['question'] = df['question'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [9]:
VALIDATION_SPLIT=0.20

## LSTM

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=30

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

In [13]:
data=df.copy()

In [14]:
print(data['type'].value_counts())

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(data['question'].values)
X = tokenizer.texts_to_sequences(data['question'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

what           609
who            402
unknown        272
affirmation    104
when            96
Name: type, dtype: int64


In [15]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Y = data['type']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)


# split the data into a training set and a validation set
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X.shape[0])

x_train = X[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Found 3685 unique tokens.
Shape of data tensor: (1483, 30)
Shape of label tensor: (1483, 5)


In [17]:
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [18]:
EMBEDDING_DIM=100

In [19]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [20]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [21]:
lstm_out = 196

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(lstm_out, dropout_U=0.25, dropout_W=0.25))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, recurrent_dropout=0.25, dropout=0.25)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           368600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               232848    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 985       
Total params: 602,433
Trainable params: 233,833
Non-trainable params: 368,600
_________________________________________________________________
None


In [27]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_val, y_val))

Train on 1187 samples, validate on 296 samples
Epoch 1/20
1187/1187 [==============================] - 1s 953us/step - loss: 0.1601 - acc: 0.9570 - val_loss: 0.0813 - val_acc: 0.9797
Epoch 2/20
1187/1187 [==============================] - 0s 259us/step - loss: 0.1505 - acc: 0.9638 - val_loss: 0.0812 - val_acc: 0.9797
Epoch 3/20
1187/1187 [==============================] - 0s 259us/step - loss: 0.1453 - acc: 0.9596 - val_loss: 0.0909 - val_acc: 0.9730
Epoch 4/20
1187/1187 [==============================] - 0s 253us/step - loss: 0.1611 - acc: 0.9520 - val_loss: 0.1144 - val_acc: 0.9730
Epoch 5/20
1187/1187 [==============================] - 0s 254us/step - loss: 0.1465 - acc: 0.9612 - val_loss: 0.0829 - val_acc: 0.9764
Epoch 6/20
1187/1187 [==============================] - 0s 257us/step - loss: 0.1383 - acc: 0.9646 - val_loss: 0.0903 - val_acc: 0.9797
Epoch 7/20
1187/1187 [==============================] - 0s 258us/step - loss: 0.1000 - acc: 0.9789 - val_loss: 0.0919 - val_acc: 0.9730
E

In [30]:
model.save_weights('anjana.h5')

In [22]:
model.load_weights('anjana.h5')

In [25]:
example = tokenizer.texts_to_sequences(["What time does the train leave"])
example = pad_sequences(example, maxlen=MAX_SEQUENCE_LENGTH)

In [26]:
le.inverse_transform(np.argmax(model.predict(example)))

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


'when'